<a href="https://colab.research.google.com/github/Boamah-Powers/ml-learning/blob/main/neural_nets_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Neural Nets

Experimenting with neural nets